In [1]:
# Importacion de librerias necesarias
import os
import gymnasium   #
import DemoC_RL   # Entorno personalizado
import numpy as np
import matplotlib.pyplot as plt
import stable_baselines3   # Biblioteca RL
import torch as th   # para modelo CNN
import torch.nn as nn
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env # Verificar el entorno
from stable_baselines3.common.vec_env import VecFrameStack, VecNormalize # Envoltorio para imagen
from stable_baselines3.sac.policies import MlpPolicy, CnnPolicy # Arquitectura de RN
from stable_baselines3 import A2C, TD3, DQN, DDPG, PPO, SAC, HER # algoritmo de entrenamiento
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.utils import get_system_info
from stable_baselines3.common.noise import NormalActionNoise

from stable_baselines3.common.monitor import Monitor

from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common import results_plotter
#get_system_info(True)

OSError: [WinError 127] No se encontró el proceso especificado. Error loading "C:\Users\USUARIO\anaconda3\envs\IAGen\Lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.

In [2]:
# Clase para guardar el mejor modelo entrenado en base a las recompensas
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """

    def __init__(self, check_freq: int, log_dir: str, verbose=1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, "best_model")
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

            # Retrieve training reward
            x, y = ts2xy(load_results(self.log_dir), "timesteps")
            if len(x) > 0:
                # Mean training reward over the last 100 episodes
                mean_reward = np.mean(y[-100:])
                if self.verbose > 0:
                    print(f"Num timesteps: {self.num_timesteps}")
                    print(
                        f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}"
                    )

                # New best model, you could save the agent here
                if mean_reward > self.best_mean_reward:
                    self.best_mean_reward = mean_reward
                    # Example for saving best model
                    if self.verbose > 0:
                        print(f"Saving new best model to {self.save_path}.zip")
                    self.model.save(self.save_path)

        return True

In [3]:
# Create log dir
log_dir = "./logs"
os.makedirs(log_dir, exist_ok=True)
# Inicializar el entorno
ENV_NAME = "DemoC_RL/TareaRL-v0"
#env = gymnasium.make(ENV_NAME) 
env = make_vec_env(ENV_NAME, n_envs=1, seed=1, monitor_dir=log_dir)
callback = SaveOnBestTrainingRewardCallback(check_freq=20, log_dir=log_dir, verbose=1)
# check_env(env, warn=True) # si funciona
# env = VecFrameStack(env, n_stack=4)

Conectado al servidor API remoto


In [4]:
policy_kwargs = dict(activation_fn=th.nn.ReLU, net_arch=dict(pi=[256, 512, 256], vf=[256, 512, 256]))

In [5]:
# Definir y entrenar el agente con A2C
model = A2C("MlpPolicy", env, gamma=0.99, policy_kwargs= policy_kwargs, verbose=1, device="auto", tensorboard_log='./A2C_tensorboard/')
model.learn(total_timesteps = 1000, callback=callback)
model.save("modeloA2C-1k") # Guardar modelo DRL

Using cuda device
Logging to ./A2C_tensorboard/A2C_1
--------------------------------------------------------------------
STEP:  1
Acciones:  [0.6478558  0.26972967 1.        ]
Distancia: 27.552  cm
--- RECOMPENSA TOTAL: -2.755 --- 
[ 0.6        -0.05        0.03750009]
--------------------------------------------------------------------
STEP:  2
Acciones:  [ 0.43944964  0.38545585 -0.44723552]
Distancia: 27.258  cm
--- RECOMPENSA TOTAL: -2.726 --- 
[ 0.6        -0.05        0.03750009]
--------------------------------------------------------------------
STEP:  3
Acciones:  [0.24862342 0.11055297 0.29628572]
Distancia: 27.303  cm
--- RECOMPENSA TOTAL: -2.730 --- 
[ 0.6        -0.05        0.03750009]
--------------------------------------------------------------------
STEP:  4
Acciones:  [ 1.        -1.        -0.7612047]
Distancia: 27.472  cm
--- RECOMPENSA TOTAL: -2.747 --- 
[ 0.6        -0.05        0.03750008]
--------------------------------------------------------------------
STE

Distancia: 27.195  cm
--- RECOMPENSA TOTAL: -2.719 --- 
[ 0.6        -0.05        0.03750009]
--------------------------------------------------------------------
STEP:  39
Acciones:  [0.6424906 0.679017  1.       ]
Distancia: 27.114  cm
--- RECOMPENSA TOTAL: -2.711 --- 
[ 0.6        -0.05        0.03750009]
--------------------------------------------------------------------
STEP:  40
Acciones:  [-0.61152965  1.          0.22535883]
Distancia: 26.743  cm
--- RECOMPENSA TOTAL: -2.674 --- 
[ 0.6        -0.05        0.03750009]
--------------------------------------------------------------------
STEP:  41
Acciones:  [-1.         -1.         -0.93648994]
Distancia: 26.988  cm
--- RECOMPENSA TOTAL: -2.699 --- 
[ 0.6        -0.05        0.03750009]
--------------------------------------------------------------------
STEP:  42
Acciones:  [ 0.05953848 -0.13699171 -1.        ]
Distancia: 26.819  cm
--- RECOMPENSA TOTAL: -2.682 --- 
[ 0.6        -0.05        0.03750009]
------------------------

--------------------------------------------------------------------
STEP:  76
Acciones:  [-0.23076472 -0.7254045   0.65528977]
Distancia: 30.396  cm
--- RECOMPENSA TOTAL: -3.040 --- 
[ 0.6        -0.05        0.03750009]
--------------------------------------------------------------------
STEP:  77
Acciones:  [-0.5879924   0.30174696  1.        ]
Distancia: 30.504  cm
--- RECOMPENSA TOTAL: -3.050 --- 
[ 0.6        -0.05        0.03748661]
--------------------------------------------------------------------
STEP:  78
Acciones:  [0.60464543 0.678228   0.0365289 ]
Distancia: 30.203  cm
--- RECOMPENSA TOTAL: -3.020 --- 
[ 0.6        -0.05        0.03750009]
--------------------------------------------------------------------
STEP:  79
Acciones:  [-0.78336555 -1.          0.38600436]
Distancia: 30.749  cm
--- RECOMPENSA TOTAL: -3.075 --- 
[ 0.6        -0.05        0.03750009]
--------------------------------------------------------------------
STEP:  80
Acciones:  [-0.22144985 -0.69970363 

Distancia: 28.881  cm
--- RECOMPENSA TOTAL: -2.888 --- 
[ 0.6        -0.05        0.03750009]
--------------------------------------------------------------------
STEP:  114
Acciones:  [-0.505039   -0.5320429  -0.00513135]
Distancia: 29.117  cm
--- RECOMPENSA TOTAL: -2.912 --- 
[ 0.6        -0.05        0.03750009]
--------------------------------------------------------------------
STEP:  115
Acciones:  [-1.          0.42349008 -0.86354446]
Distancia: 28.743  cm
--- RECOMPENSA TOTAL: -2.874 --- 
[ 0.6        -0.05        0.03750009]
--------------------------------------------------------------------
STEP:  116
Acciones:  [ 0.44296297  0.37694335 -0.08979465]
Distancia: 28.552  cm
--- RECOMPENSA TOTAL: -2.855 --- 
[ 0.6        -0.05        0.03750009]
--------------------------------------------------------------------
STEP:  117
Acciones:  [ 0.61968595 -0.6779586   0.56305534]
Distancia: 28.980  cm
--- RECOMPENSA TOTAL: -2.898 --- 
[ 0.6        -0.05        0.03750009]
--------------

Distancia: 25.957  cm
--- RECOMPENSA TOTAL: -2.596 --- 
[ 0.6        -0.05        0.03750009]
--------------------------------------------------------------------
STEP:  151
Acciones:  [-1.         -0.28578776 -0.9130422 ]
Distancia: 25.926  cm
--- RECOMPENSA TOTAL: -2.593 --- 
[ 0.6        -0.05        0.03749893]
--------------------------------------------------------------------
STEP:  152
Acciones:  [-0.64872223  0.96733963  1.        ]
Distancia: 25.647  cm
--- RECOMPENSA TOTAL: -2.564 --- 
[ 0.6        -0.05        0.03750009]
--------------------------------------------------------------------
STEP:  153
Acciones:  [ 1.  1. -1.]
Distancia: 25.003  cm
--- RECOMPENSA TOTAL: -2.500 --- 
[ 0.6        -0.05        0.03748661]
--------------------------------------------------------------------
STEP:  154
Acciones:  [ 0.37924254  0.92224807 -0.357637  ]
Distancia: 24.512  cm
--- RECOMPENSA TOTAL: -2.451 --- 
[ 0.6        -0.05        0.03750009]
--------------------------------------

...

--------------------------------------------------------------------
STEP:  839
Acciones:  [-0.73185354 -1.         -0.14845982]
Distancia: 101050.393  cm
--- RECOMPENSA TOTAL: -10105.039 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  840
Acciones:  [ 0.34122866  1.         -0.98295385]
Distancia: 101049.901  cm
--- RECOMPENSA TOTAL: -10104.990 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  841
Acciones:  [-0.5547389  -0.4286049   0.49590364]
Distancia: 101050.147  cm
--- RECOMPENSA TOTAL: -10105.015 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  842
Acciones:  [-1.         -0.48779237  1.        ]
Distancia: 101050.649  cm
--- RECOMPENSA TOTAL: -10105.065 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
----------------------------------------

Distancia: 101052.997  cm
--- RECOMPENSA TOTAL: -10105.300 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  874
Acciones:  [ 1.         -1.          0.14723645]
Distancia: 101053.073  cm
--- RECOMPENSA TOTAL: -10105.307 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  875
Acciones:  [ 0.4834368   0.23540549 -0.08572277]
Distancia: 101053.030  cm
--- RECOMPENSA TOTAL: -10105.303 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  876
Acciones:  [-0.5763846  -1.          0.14378254]
Distancia: 101053.100  cm
--- RECOMPENSA TOTAL: -10105.310 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  877
Acciones:  [-1.         1.        -0.9035821]
Distancia: 101052.646  cm
--- RECOMPENSA TO

Distancia: 101056.068  cm
--- RECOMPENSA TOTAL: -10105.607 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  909
Acciones:  [-0.7475568  1.         0.659516 ]
Distancia: 101056.395  cm
--- RECOMPENSA TOTAL: -10105.639 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  910
Acciones:  [-0.19873129  1.          0.17127201]
Distancia: 101056.477  cm
--- RECOMPENSA TOTAL: -10105.648 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  911
Acciones:  [ 0.4070196  -0.06219627  1.        ]
Distancia: 101056.978  cm
--- RECOMPENSA TOTAL: -10105.698 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  912
Acciones:  [0.8028702  0.5061899  0.61475164]
Distancia: 101057.281  cm
--- RECOMPENSA TOTAL

--------------------------------------------------------------------
STEP:  943
Acciones:  [ 0.05828139 -0.01076344  0.8896479 ]
Distancia: 101053.629  cm
--- RECOMPENSA TOTAL: -10105.363 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  944
Acciones:  [-1.        -0.9699459  1.       ]
Distancia: 101054.133  cm
--- RECOMPENSA TOTAL: -10105.413 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  945
Acciones:  [ 0.42479235 -0.1356047  -0.7212767 ]
Distancia: 101053.774  cm
--- RECOMPENSA TOTAL: -10105.377 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  946
Acciones:  [ 0.11263943 -0.6240554  -1.        ]
Distancia: 101053.275  cm
--- RECOMPENSA TOTAL: -10105.328 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
-------------------------------------------

Distancia: 101046.783  cm
--- RECOMPENSA TOTAL: -10104.678 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  978
Acciones:  [1.         0.22019434 0.98884183]
Distancia: 101047.276  cm
--- RECOMPENSA TOTAL: -10104.728 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  979
Acciones:  [-1.  1.  1.]
Distancia: 101047.774  cm
--- RECOMPENSA TOTAL: -10104.777 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  980
Acciones:  [-0.42452493  1.         -1.        ]
Distancia: 101047.271  cm
--- RECOMPENSA TOTAL: -10104.727 --- 
[ 9.5774192e-01  6.3808994e+00 -1.0108930e+03]
--------------------------------------------------------------------
STEP:  981
Acciones:  [-0.6138444  -0.06781127  1.        ]
Distancia: 101047.772  cm
--- RECOMPENSA TOTAL: -10104.777 --- 
[ 9

In [10]:
# Graficar Curva de Aprendizaje obtenida durante entrenamiento
# Helper from the library
results_plotter.plot_results(
    [log_dir], 1000, results_plotter.X_TIMESTEPS, "Datos de entrenamiento"
)
def moving_average(values, window):
    """
    Smooth values by doing a moving average
    :param values: (numpy array)
    :param window: (int)
    :return: (numpy array)
    """
    weights = np.repeat(1.0, window) / window
    return np.convolve(values, weights, "valid")


def plot_results(log_folder, title="Curva de aprendizaje"):
    """
    plot the results

    :param log_folder: (str) the save location of the results to plot
    :param title: (str) the title of the task to plot
    """
    x, y = ts2xy(load_results(log_folder), "timesteps")
    y = moving_average(y, window=100)
    # Truncate x
    x = x[len(x) - len(y) :]

    fig = plt.figure(title)
    plt.figure(figsize=(8,2), dpi=500)
    plt.plot(x, y)
    plt.grid()
    plt.xlabel("Number of Timesteps",fontsize=7)
    plt.ylabel("Rewards",fontsize=7)
    plt.title(title + " Smoothed",fontsize=8)
    plt.tick_params(axis='both', which='major', labelsize=7)
    plt.show()


plot_results(log_dir)



IndexError: index -1 is out of bounds for axis 0 with size 0

<Figure size 800x200 with 0 Axes>